In [1]:
import os
import numpy as np
import hashlib
import torchvision.models as models
from torch import nn
from PIL import Image
from collections import Counter, defaultdict

# Working with Images Lab
## Information retrieval, preprocessing, and feature extraction

In this lab, you'll work with images of felines (cats), which have been classified according to their taxonomy. Each subfolder contains images of a particular species. The dataset is located [here](https://www.kaggle.com/datasets/datahmifitb/felis-taxonomy-image-classification) but it's also provided to you in the `data/` folder.

### Problem 1. Some exploration (1 point)
How many types of cats are there? How many images do we have of each? What is a typical image size? Are there any outliers in size?

In [2]:
dataset_path = "data/"
species = [d for d in os.listdir(dataset_path)]

print(f"Number of cat types (species): {len(species)}")
print("Species names:", species)

Number of cat types (species): 7
Species names: ['african-wildcat', 'blackfoot-cat', 'chinese-mountain-cat', 'domestic-cat', 'european-wildcat', 'jungle-cat', 'sand-cat']


In [3]:
for s in species:
    folder = os.path.join(dataset_path, s)
    count = len([f for f in os.listdir(folder)])
    print(f"{s}: {count} images")

african-wildcat: 91 images
blackfoot-cat: 79 images
chinese-mountain-cat: 40 images
domestic-cat: 64 images
european-wildcat: 57 images
jungle-cat: 86 images
sand-cat: 73 images


In [4]:
image_sizes = []

for s in species:
    folder = os.path.join(dataset_path, s)
    
    for fname in os.listdir(folder):
        path = os.path.join(folder, fname)

        if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            with Image.open(path) as img:
                image_sizes.append(img.size)

most_common_size = Counter(image_sizes).most_common(1)[0]
print(f"Most common image size: {most_common_size[0]} (occurs {most_common_size[1]} times)")

widths, heights = zip(*image_sizes)
avg_width = sum(widths) / len(widths)
avg_height = sum(heights) / len(heights)
print(f"Average image size: {avg_width:.2f} x {avg_height:.2f}")

Most common image size: (275, 183) (occurs 108 times)
Average image size: 376.10 x 290.52


Let's define outliers as images much smaller or much larger than typical (mean +- 2 * std).

In [5]:
widths_np_array = np.array(widths)
heights_np_array = np.array(heights)

mean_widths, std_widths = widths_np_array.mean(), widths_np_array.std()
mean_heights, std_heights = heights_np_array.mean(), heights_np_array.std()

outliers = [(w, h) for (w, h) in image_sizes if
            w < mean_widths - 2 * std_widths or w > mean_widths + 2 * std_widths or
            h < mean_heights - 2 * std_heights or h > mean_heights + 2 * std_heights]

print(f"Found {len(outliers)} image size outliers:")

if outliers:
    print("Some outlier sizes:", outliers[:5])

Found 25 image size outliers:
Some outlier sizes: [(1500, 1038), (4000, 3000), (4000, 3000), (2048, 1365), (1644, 2598)]


### Problem 2. Duplicat(e)s (1 point)
Find a way to filter out (remove) identical images. I would recommnend using file hashes, but there are many approaches. Keep in mind that during file saving, recompression, etc., a lot of artifacts can change the file content (bytes), but not visually.

In [6]:
def get_file_hash(path):
    """
    Function to obtain file hash receiving a file
    """
    
    with open(path, 'rb') as f:
        return hashlib.md5(f.read()).hexdigest()

In [7]:
hashes = defaultdict(list)

for s in species:
    folder = os.path.join(dataset_path, s)
    
    for fname in os.listdir(folder):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            path = os.path.join(folder, fname)
            file_hash = get_file_hash(path)
            hashes[file_hash].append(path)

In [8]:
duplicates = [paths for paths in hashes.values() if len(paths) > 1]
print(f"Found {len(duplicates)} groups of exact duplicates.")

Found 0 groups of exact duplicates.


In [9]:
for group in duplicates:
    for dup in group:
        os.remove(dup)
        print(f"Removed: {dup}")

I used MD5 file hashing to detect and remove duplicate images. This method works well for exact file duplicates, where the image content and encoding are identical. However, it may miss visually identical images that differ in compression or format. I could use perceptual hashing (like imagehash.phash()), which compares visual content rather than file bytes.

### Problem 3. Loading a model (2 points)
Find a suitable, trained convolutional neural network classifier. I recommend `ResNet50` as it's small enough to run well on any machine and powerful enough to make reasonable predictions. Most ready-made classifiers have been trained for 1000 classes.

You'll need to install libraries and possibly tinker with configurations for this task. When you're done, display the total number of layers and the total number of parameters. For ResNet50, you should expect around 50 layers and 25M parameters.

In [10]:
resnet = models.resnet50()

total_params = sum(p.numel() for p in resnet.parameters())
main_layers = [m for m in resnet.modules() if isinstance(m, (nn.Conv2d, nn.Linear))]

print(f"Total parameters: {total_params:,}")
# We need to count only layers with weights (Conv2d, Linear) -> main layers
print(f"Total main layers: {len(main_layers)}")

Total parameters: 25,557,032
Total main layers: 54


### Problem 4. Prepare the images (1 point)
You'll need to prepare the images for passing to the model. To do so, they have to be resized to the same dimensions. Most available models have a specific requirement for sizes. You may need to do additional preprocessing, depending on the model requirements. These requirements should be easily available in the model documentation.

### Problem 5. Load the images efficiently (1 point)
Now that you've seen how to prepare the images for passing to the model... find a way to do it efficiently. Instead of loading the entire dataset in the RAM, read the images in batches (e.g. 4 images at a time). The goal is to read these, preprocess them, maybe save the preprocessed results in RAM.

If you've already done this in one of the previous problems, just skip this one. You'll get your point for it.

\* Even better, save the preprocessed image arrays (they will not be valid .jpg file) as separate files, so you can load them "lazily" in the following steps. This is a very common optimization to work with large datasets.

### Problem 6. Predictions (1 point)
Finally, you're ready to get into the meat of the problem. Obtain predictions from your model and evaluate them. This will likely involve manual work to decide how the returned classes relate to the original ones.

Create a [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) to evaluate the classification.

### Problem 7. Grayscale (1 point)
Converting the images to grayscale should affect the classification negatively, as we lose some of the color information.

Find a way to preprocess the images to grayscale (using what you already have in Problem 4 and 5), pass them to the model, and compare the classification results to the previous ones.

### Problem 8. Deep image features (1 point)
Find a way to extract one-dimensional vectors (features) for each (non-grayscale) image, using your model. This is typically done by "short-circuiting" the model output to be an intermediate layer, while keeping the input the same. 

In case the outputs (also called feature maps) have different shapes, you can flatten them in different ways. Try to not create huge vectors; the goal is to have a relatively short sequence of numbers which describes each image.

You may find a tutorial like [this](https://towardsdatascience.com/exploring-feature-extraction-with-cnns-345125cefc9a) pretty useful but note your implementation will depend on what model (and framework) you've decided to use.

It's a good idea to save these as one or more files, so you'll spare yourself a ton of preprocessing.

### Problem 9. Putting deep image features to use (1 points)
Try to find similar images, using a similarity metric on the features you got in the previous problem. Two good metrics are `mean squared error` and `cosine similarity`. How do they work? Can you spot images that look too similar? Can you explain why?

\* If we were to take Fourier features (in a similar manner, these should be a vector of about the same length), how do they compare to the deep features; i.e., which features are better to "catch" similar images?

### * Problem 10. Explore, predict, and evaluate further
You can do a ton of things here, at your desire. For example, how does masking different areas of the image affect classification - a method known as **saliency map** ([info](https://en.wikipedia.org/wiki/Saliency_map))? Can we detect objects? Can we significantly reduce the number of features (keeping the quality) that we get? Can we reliably train a model to predict our own classes? We'll look into these in detail in the future.